In [1]:
!git clone https://github.com/aub-mind/arabert.git
!pip install simpletransformers
!pip install farasapy
!pip install pyarabic

Cloning into 'arabert'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 564 (delta 25), reused 22 (delta 22), pack-reused 535
Receiving objects: 100% (564/564), 9.11 MiB | 26.65 MiB/s, done.
Resolving deltas: 100% (326/326), done.
     |████████████████████████████████| 249 kB 8.6 MB/s 
     |████████████████████████████████| 1.2 MB 63.4 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 342 kB 66.7 MB/s 
     |████████████████████████████████| 1.8 MB 51.3 MB/s 
     |████████████████████████████████| 6.6 MB 53.0 MB/s 
     |████████████████████████████████| 10.1 MB 51.2 MB/s 
     |████████████████████████████████| 4.2 MB 53.1 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 596 kB 64.4 MB/s 
     |████████████████████████████████| 181 kB 71.8 MB/s 
     |████████████████████████████

     |████████████████████████████████| 126 kB 7.1 MB/s 


In [2]:
!pip install simpletransformers
!pip install setuptools==57.4.0

In [3]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import torch
import json, argparse
from collections import Counter
import re
import string
import logging
import os
import numpy as np
import pandas as pd
from farasa.segmenter import FarasaSegmenter
farasa_segmenter = FarasaSegmenter()
model_name = 'aubmindlab/bert-base-arabertv02'
model_type = 'bert'

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


100%|██████████| 241M/241M [00:24<00:00, 10.0MiB/s]


In [4]:
# # Load data 

# import requests

# # Get JSON string data from Git hub using web requests library
# training data = requests.get("https://raw.githubusercontent.com/OmarAbuhassan/Question_answering/main/train.json")


In [5]:
# Checking the Google Colab using cuda instead of cpu.

if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [6]:
from arabert.preprocess import ArabertPreprocessor


arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوترً المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text), farasa_segmenter.segment(text)

('ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري',
 'و+لن نبالغ إذا قل+نا إن هاتف أو كمبيوتر ال+مكتب في زمن+نا هذا ضروري')

In [7]:
def load(file_name,training = False):
  with open(file_name) as f:
    dataset = json.load(f)['data']

  data = []
  for article in dataset:
    for paragraph in article['paragraphs']:
      data.append(paragraph)

  return data


In [8]:
train = load('train.json')
test = load('test.json')

In [9]:
train[0]

{'context': 'السعودية أو (رسميًا: المملكة العربية السعودية) هي دولة عربية، وتعد أكبر دولة في الشرق الأوسط وتقع تحديدًا في الجنوب الغربي من قارة آسيا وتشكل الجزء الأكبر من شبه الجزيرة العربية إذ تبلغ مساحتها حوالي مليوني كيلومتر مربع.',
 'qas': [{'answers': [{'answer_start': 21,
     'text': 'المملكة العربية السعودية)'}],
   'id': '333772766499',
   'question': ' - أي دولة هي أكبر دولة في الشرق الأوسط؟ ال'},
  {'answers': [{'answer_start': 109, 'text': 'الجنوب الغربي'}],
   'id': '262204981583',
   'question': ' - أين تقع المملكة العربية السعودية في آسيا؟ ال'},
  {'answers': [{'answer_start': 194, 'text': 'حوالي مليوني كيلومتر مربع.'}],
   'id': '809283218984',
   'question': ' - ما هي مساحة الجزء الأكبر من شبه الجزيرة العربية؟ ال'}]}

In [10]:
# Configure the model
models_args = QuestionAnsweringArgs()
models_args.train_batch_size = 4
models_args.evaluate_during_training = True
models_args.n_best_size = 3
models_args.save_model_every_epoch = False
models_args.save_steps = -1
models_args.num_train_epochs = 5
models_args.manual_seed = 109
models_args.use_early_stopping = True
models_args.output_dir = "/content/output/"

In [11]:
### Remove output folder
!rm -rf outputs
!rm -rf output

In [12]:
# Instatiate the model and set the "args" to model_args variable

model = QuestionAnsweringModel(
    model_type, model_name, args= models_args
)

Downloading:   0%|          | 0.00/384 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized fr

Downloading:   0%|          | 0.00/805k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

In [13]:
import sklearn
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(train)

In [14]:
# torch.cuda.empty_cache()
model.train_model(train_data=train_data, eval_data = eval_data, acc=sklearn.metrics.accuracy_score)

add example index and unique id: 100%|██████████| 498/498 [00:00<00:00, 339581.11it/s]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/126 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)

convert squad examples to features: 100%|██████████| 168/168 [00:00<00:00, 184.64it/s]

add example index and unique id: 100%|██████████| 168/168 [00:00<00:00, 428354.45it/s]


Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/126 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 168/168 [00:00<00:00, 188.64it/s]

add example index and unique id: 100%|██████████| 168/168 [00:00<00:00, 396089.42it/s]


Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

(630,
 {'acc': [0.07738095238095238,
   0.16666666666666666,
   0.17261904761904762,
   0.1488095238095238,
   0.14285714285714285],
  'correct': [13, 28, 29, 25, 24],
  'eval_loss': [-2.6961495535714284,
   -4.1223028273809526,
   -5.330171130952381,
   -6.247023809523809,
   -6.874813988095238],
  'global_step': [126, 252, 378, 504, 630],
  'incorrect': [25, 38, 38, 37, 38],
  'similar': [130, 102, 101, 106, 106],
  'train_loss': [2.349273681640625,
   1.779022216796875,
   1.0981330871582031,
   0.46985626220703125,
   0.19824635982513428]})

In [15]:
result, predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score )

add example index and unique id: 100%|██████████| 639/639 [00:00<00:00, 621414.39it/s]


Running Evaluation:   0%|          | 0/81 [00:00<?, ?it/s]

In [16]:
result

{'acc': 0.24100156494522693,
 'correct': 154,
 'eval_loss': -6.764130015432099,
 'incorrect': 139,
 'similar': 346}

In [17]:
predictions["incorrect_text"]

{'101765027998': {'predicted': 'يرى دابّةً بيضاء',
  'question': 'ماذا رأى الرسول حين أخرجهجبريل إلى باب المسجد؟',
  'truth': 'دابّةً بيضاء بين البغل والحمار،'},
 '105673018704': {'predicted': 'يوم كيبور) كما تعرف في إسرائيل، هي حرب شنتها كل من مصر وسوريا على إسرائيل عام 1973',
  'question': 'بماذا تسمي حرب اكتوبر في اسرائيل؟',
  'truth': 'حرب يوم الغفران'},
 '106366036897': {'predicted': 'واللون الأخضر',
  'question': 'ما هو سر محبته للجمال و الالوان وبالذات اللون الاخضر؟',
  'truth': 'منزلهم الدمشقي كان لديهم أغلب أصناف الزروع الشاميّة من زنبق وريحان وياسمين ونعناع ونارنج.'},
 '112079412992': {'predicted': 'الكندي',
  'question': 'ما جنسية جندح بن حجر؟',
  'truth': 'هو شاعرعربي'},
 '11250133938': {'predicted': '500 مليون ناطق بالإسبانية،',
  'question': 'كم هناك من ناطق بالغة الإسبانية؟',
  'truth': 'أكثر من 500 مليون'},
 '116930628760': {'predicted': 'تفكك الدولة العباسية وتناثر الدويلات الإسلامية التي قامت على أنقاضها.',
  'question': 'بماذا تتميز الفترة التي نشأ فيها أبو الطيب الم

In [18]:
predictions["similar_text"]

{'100269981551': {'predicted': '9 بطولات',
  'question': 'كم عدد البطولات اللتى حققها زيدان مع النادى الملكى؟',
  'truth': '9 بطولات في عامين ونصف'},
 '102861223894': {'predicted': 'من 1919',
  'question': 'متى تم إنشاء منظمة عصبة الأمم؟',
  'truth': '1919'},
 '104499435818': {'predicted': 'العام 1505',
  'question': 'متي تم اكتشاف موريشيوس؟',
  'truth': 'في العام 1505'},
 '110058198329': {'predicted': 'القرآن أو القرآن الكريم',
  'question': 'ما هو الكتاب الرئيسى فى الاسلام؟',
  'truth': 'القرآن الكريم'},
 '111186060264': {'predicted': '',
  'question': 'ما هي التغيرات التي سببتها الثوره الفرنسيه عندما استوحت افكار ليبراليه وراديكاليه؟',
  'truth': 'بشكل عميق مسار التاريخ الحديث، وأطلقت الانحدار العالمي للملكيات المطلق واستبدالها بالجمهوريات.'},
 '114069415149': {'predicted': '120 كيلومترا',
  'question': 'كم تبعد مكة المكرمة عن الطائف؟',
  'truth': 'حوالي 120 كيلومترا في الاتجاه الشرقي،'},
 '118072267433': {'predicted': '',
  'question': 'في أي عام إنتقلت أم كلثوم و والدها إلى القاهر

In [19]:
predictions["correct_text"]

{'105236247076': 'وفي أكتوبر 2009',
 '10737648493': 'الاتحاد الدولي لكرة القدم (الـفيفا).',
 '109564956217': '«دولة الرباط والإصلاح».',
 '113607516207': 'لمؤتمر دومبارتون أوكس الذي عقد في العاصمة واشنطن.',
 '121591221310': 'بالتشبه بأحمد زكي وتقليده،',
 '127784589749': '4,675,710 ملايين نسمة.',
 '128289817127': 'رئيس المخابرات الحربية والاستطلاع.',
 '131191283668': 'وفقاً لمكان وزمان نزول الوحي بها.',
 '137536264899': 'لاعب كرة قدم فرنسي من أصل جزائري،',
 '138768012694': 'في القرن السادس الهجري',
 '139810062449': 'في عام 1535،',
 '145022408477': 'ثلاثمائة وبضعة عشر رجلاً،',
 '147293444932': 'تقع غرب المملكة العربية السعودية،',
 '153159225042': 'Sultan Süleyman-ı Evvel)',
 '156381511284': '20%',
 '164335882851': 'ثلاث آيات',
 '168251134990': '27 محافظة',
 '180300657120': 'ثورات وفتناً كثيرة،',
 '188260747433': 'الخطيب التبريزي',
 '195740200624': 'نظام ملكي دستوري',
 '201991055920': 'الخليفة عمر بن الخطاب',
 '211558754598': '30 آية.',
 '214222340937': 'حوالي ثلاثة ملايين دولار.',
 '21569

In [20]:
predictions, raw_outputs = model.predict(test)

add example index and unique id: 100%|██████████| 639/639 [00:00<00:00, 683714.35it/s]


Running Prediction:   0%|          | 0/81 [00:00<?, ?it/s]

Obtain metrics EM and F1 on our data set. This follows the offical SQuAD 1.1 evaluation script

In [21]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import nltk
import random
nltk.download('punkt')
from random import randint

test = load('test.json')
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    
    def remove_articles_ar(text):
        return re.sub('\sال^|ال', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles_ar(remove_articles(remove_punc(lower(s)))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, predictions, ground_truth):
    scores_for_ground_truths = []
    for prediction in predictions:
        score = metric_fn(prediction, ground_truth)
        print("prediction:", prediction, "   --score: ", score)        
        scores_for_ground_truths.append(score)
    
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = exact_sentence = inclusion = random = 0
    for article in dataset:
        print("context: ",article["context"])
        for qa in article['qas']:
            total += 1
            print("question: ",qa["question"])
            
            ground_truth = {"id": qa["id"],'answer':[qa['answers'][0]["text"]]}
            answer_gt = ground_truth["answer"][0]
            prediction = next(item for item in predictions if item["id"] == ground_truth["id"])
            answers_pred = prediction["answer"]
            
            if answers_pred == "empty": answers_pred = " "

            print("\n","gt:", answer_gt)

            exact_match += metric_max_over_ground_truths(exact_match_score, answers_pred, answer_gt)
            f1 += metric_max_over_ground_truths(f1_score, answers_pred, answer_gt)
            print("------------------ \n")
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [22]:
test[0]

{'context': 'حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر، وهو خير أعمامه لقوله: «خَيْرُ إِخْوَتِي عَلِيٌّ، وَخَيْرُ أَعْمَامِي حَمْزَةُ رَضِيَ اللَّهُ عَنْهُمَا».',
 'qas': [{'answers': [{'answer_start': 34,
     'text': 'صحابي من صحابة رسول الإسلام محمد، وعمُّه وأخوه من الرضاعة وأحد وزرائه الأربعة عشر،'}],
   'id': '621723207492',
   'question': 'من هو حمزة بن عبد المطلب؟'},
  {'answers': [{'answer_start': 166, 'text': 'وَخَيْرُ أَعْمَامِي'}],
   'id': '189105393656',
   'question': 'بما وصفه رسول الله؟'},
  {'answers': [{'answer_start': 139, 'text': '«خَيْرُ إِخْوَتِي عَلِيٌّ،'}],
   'id': '662616978980',
   'question': 'بما وصف رسول الله على ؟'}]}

In [23]:
evaluate(test, predictions)

Streaming output truncated to the last 5000 lines.
------------------ 

question:  من هم طرفي النزاع في معركة اليرموك؟

 gt: المسلمين والروم
prediction: المسلمين والروم    --score:  True
prediction: بين المسلمين والروم    --score:  False
prediction: المسلمين والروم (الإمبراطورية البيزنطية)،    --score:  False
prediction: المسلمين والروم    --score:  1.0
prediction: بين المسلمين والروم    --score:  0.8571428571428571
prediction: المسلمين والروم (الإمبراطورية البيزنطية)،    --score:  0.7499999999999999
------------------ 

question:  لماذا يعتبر بعض المؤرخين معركة اليرموك من أهم معارك العالم؟

 gt: لأنها كانت بداية أول موجة انتصارات للمسلمين خارج جزيرة العرب،
prediction: لأنها    --score:  False
prediction: ويعتبرها بعض المؤرخين من أهم المعارك في تاريخ العالم    --score:  False
prediction: ويعتبرها بعض المؤرخين من أهم المعارك في تاريخ العالم لأنها    --score:  False
prediction: لأنها    --score:  0.18181818181818182
prediction: ويعتبرها بعض المؤرخين من أهم المعارك في تاريخ العالم    --sc

{'exact_match': 39.749608763693274, 'f1': 60.31057950275415}

# Predict

In [24]:
to_predict = [
    {
        "context": "هي دولة عربية تقع في شمال شرق أفريقيا تحدها من الشرق إثيوبيا وإريتريا ومن الشمال مصر وليبيا ومن الغرب تشاد وجمهورية أفريقيا الوسطى ومن الجنوب دولة جنوب السودان.",
        "qas": [
            {
                "question": "ما الدول التي تحد السودان من الغرب ؟",
                "id": "0",
            }
        ],
    }
]

model.predict(to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 5140.08it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'answer': ['تشاد وجمهورية أفريقيا الوسطى',
    'تشاد وجمهورية أفريقيا الوسطى ومن الجنوب دولة جنوب السودان.',
    'تشاد'],
   'id': '0'}],
 [{'id': '0',
   'probability': [0.7982558400254749,
    0.11059408952975523,
    0.09115007032034973]}])